# Resumo do código

### <u>Código que gera o ficheiro OUT2_DataFusion_2022</u>
---
Faz o mesmo que o 2PilototWide só que recebe dados de 2022

---
> Input
> - Única diferença é coluna Id que deve corresponder à ordem dos dias (porque os dias homólogos não coincidem no número)

> Output
> - O mesmo de 2PilotoWide, embora nem todas as métricas sejam necessárias

In [2]:
import pandas as pd
import numpy as np

def escrever_excel(dfa, nome):
    dfa.to_excel('%s.xlsx' %nome, index=False)
    
def escrever_csv(dfa, nome):
    dfa.to_csv(nome+'.csv', index=False)

# Ler e Mudar nomes

- # `Ninjas (sinal)`

In [3]:
#Info Ninja
dfNinjas1=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\1Ninjas1fds.xlsx")
dfNinjas2=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\1Ninjas2fds.xlsx")
dfNinjas3=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\1Ninjas3fds.xlsx")
dfNinjas4=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\1Ninjas4fds.xlsx")

# Juntar dataframes no caso de serem vários ficheiros
dataframes=[dfNinjas1,dfNinjas2,dfNinjas3,dfNinjas4]
dfNinjas=pd.concat(dataframes, ignore_index=True)

#Renomear colunas
dfNinjas = dfNinjas.rename(columns={' Data de Resposta': 'DATA', "Código da loja":"STORE", "Nome da Loja":"Loja"})

> Definir variáveis importantes
> - Se houver um ficheiro com os produtos

In [4]:
# Ler ficheiro para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\produtos.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()
resto = dfNinjas.columns.difference(produtos)

> - Se se quiser usar o primeiro e o último produtos

In [143]:
# Nomes de entrada
x= input("Primeiro produto: ")                # CAFÉ DELTA Q MYTHIQ 80CAP
y= input("Último produto: ")                  # BEBIDA CEREAIS DELTA C/20%CAFE FR 200G

# Passar para uma lista
produtos = dfNinjas.columns[dfNinjas.columns.get_loc(x): dfNinjas.columns.get_loc(y)+1].tolist()
resto = dfNinjas.columns.difference(produtos)

Primeiro produto: CAFÉ DELTA Q MYTHIQ 80CAP
Último produto: Delta Cafés Mistura + café 200grs


<font color=red> Aparte </font>

---

# Estruturar a base de dados

- altura do dia

In [5]:
# Hora para datetime
dfNinjas['Hora']= pd.to_datetime(dfNinjas['Hora'], format='%H:%M:%S').dt.time  

#coluna nova para data e hora
dfNinjas['DataComp'] = pd.to_datetime(dfNinjas['DATA'].dt.strftime('%Y-%m-%d') + ' ' + dfNinjas['Hora'].astype(str)) 

In [6]:
#Definir que altura do dia é o quê
altura_do_dia = lambda x: 'Manhã' if x < 12 else ('Tarde' if x < 18 else 'Noite')

dfNinjas['Altura_do_Dia'] = dfNinjas['DataComp'].apply(lambda x: altura_do_dia(x.hour))
#dfNinjas['Altura do Dia'] = dfNinjas['DATA'].apply(lambda x: altura_do_dia(x.hour))
dfNinjas=dfNinjas.drop(columns=["DataComp"])

# Fazer o dataframe que servirá de modelo

In [7]:
import itertools

# Definir os valores para cada coluna
dates = dfNinjas["DATA"].unique()
stores = dfNinjas["STORE"].unique()

# Fazer todas as combinações de valores
combinations = list(itertools.product(dates, stores))

# Criar dataframe com essas combinações
dfModelo = pd.DataFrame(combinations, columns=['DATA', 'STORE'])

# Converter para Datetime
dfModelo['DATA'] = pd.to_datetime(dfModelo['DATA'], format='%Y-%m-%d')

In [8]:

# Definir os valores de altura do dia
times = ['Manhã', 'Tarde', 'Noite']

# Create a list of dataframes, each with a single value from times in a new column
dfs = [dfModelo.assign(Altura_do_Dia=t) for t in times]

# Concatenate the list of dataframes into a single dataframe
df_Mergir = pd.concat(dfs, ignore_index=True)


> `MERGIR`

In [9]:
dfNinjas = pd.merge(df_Mergir, dfNinjas, on=['DATA', 'STORE', 'Altura_do_Dia'], how='left')

dfNinjas = dfNinjas.sort_values(by=["STORE","DATA"])

In [10]:
# Valores nulos no nome da loja vão ser substituidos pelo primeiro valor anterior não-nulo

dfNinjas['Loja'] = dfNinjas['Loja'].fillna(method='ffill')

- Dia

In [11]:
nome_dia = dfNinjas['DATA'].dt.day_name().map({'Friday': 'Sexta', 'Saturday': 'Sábado', 'Sunday': 'Domingo'})

dfNinjas['Dia'] = nome_dia

- Semana

In [12]:
dfNinjas['Semana'] = dfNinjas['DATA'].dt.isocalendar().week

dfNinjas['Semana'] = dfNinjas.groupby('Semana').ngroup() + 1

- Completar

In [13]:
df_columns = dfNinjas.columns.tolist()
indexed_columns = [f"{index}: {column}" for index, column in enumerate(df_columns)]

In [14]:
dfNinjasOrg=dfNinjas.copy()
# Get the current column list
cols = dfNinjasOrg.columns.tolist()

# Remove the last two columns and save them
last_two_cols = cols[-2:]
cols = cols[:-2]

# Insert the last two columns to the desired index
cols.insert(5, last_two_cols[0])
cols.insert(6, last_two_cols[1])

# Reorder the columns in the DataFrame
dfNinjasOrg = dfNinjasOrg[cols]

In [15]:
df_columns = dfNinjasOrg.columns.tolist()
indexed_columns = [f"{index}: {column}" for index, column in enumerate(df_columns)]

In [17]:
df=dfNinjasOrg.copy()

# Extract the day and month components from the 'datetime' column
df['day'] = pd.to_datetime(df['DATA']).dt.day
df['month'] = pd.to_datetime(df['DATA']).dt.month

# Create a new column combining day and month
df['day_month'] = df['day'].astype(str) + '-' + df['month'].astype(str)

# Assign unique IDs to each unique day-month combination
df['id'], _ = pd.factorize(df['day_month'])

df = df.drop(columns=['day', 'month', 'day_month'])

df.head(13)

,DATA,STORE,Altura_do_Dia,Loja,Hora,Dia,Semana,CAFÉ DELTA Q MYTHIQ 80CAP,CAFÉ DELTA Q QALIDUS 80CAP,CAFÉ DELTA Q QALIDUS 10CAP,CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,CAFÉ BELLISSIMO INTENSO 200GR,CAFÉ DELTA RITUAL MU 220G,CAFÉ SOLÚVEL DELTA FRASCO 100G,CAFÉ SOLÚVEL DELTA FRASCO 200G,CEVADA SOLÚVEL DELTA FRASCO 200G,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,id
0,2023-04-14,1,Manhã,Continente Matosinhos,10:39:00,Sexta,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0
120,2023-04-14,1,Tarde,Continente Matosinhos,14:46:00,Sexta,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0
241,2023-04-14,1,Noite,Continente Matosinhos,19:08:00,Sexta,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0
10,2023-04-15,1,Manhã,Continente Matosinhos,10:56:25,Sábado,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
130,2023-04-15,1,Tarde,Continente Matosinhos,15:19:35,Sábado,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
251,2023-04-15,1,Noite,Continente Matosinhos,19:04:28,Sábado,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
20,2023-04-16,1,Manhã,Continente Matosinhos,NaN,Domingo,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
140,2023-04-16,1,Tarde,Continente Matosinhos,15:02:00,Domingo,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2
261,2023-04-16,1,Noite,Continente Matosinhos,19:20:55,Domingo,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2
30,2023-04-21,1,Manhã,Continente Matosinhos,09:30:00,Sexta,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3


In [18]:
dfNinjas=df.copy()

---

<font color=red> Fim de Aparte </font>

> Reposição

In [19]:
# Ler ficheiro
dfRepos=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\LojasReposição.xlsx")

# Criar coluna de reposição
dfNinjas['Reposição'] = [1 if val in dfRepos['Loja'].values else 0 for val in dfNinjas['Loja']]

- #  `Delta (stocks, reposição e sellouts)`

> Stocks

In [20]:
# Ler o ficheiro long com Stocks e Fornecimento
dfDelta=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\2Delta2022Id.xlsx", sheet_name="Sheet1")

# Renomear
dfDelta = dfDelta.rename(columns={"STORE_NAME":"Loja"})

In [23]:
dfDelta[dfDelta["SOH"]==0]
#dfDelta.head()

,DATA,EAN,DESC_ARTIGO,STORE,Loja,SOH,INTRANSIT,EXPECTED,PRES_STOCK,Dia_Actual,1_Dia_Antes,Reposição,Semana,Dia,id


# VER

In [153]:
#dfNinjas.head()
#dfDelta.info()

# Código para Delta (Stocks)

In [32]:
# Dataframe mergido no fim
dfMeio=dfNinjas.copy()


for coluna in produtos:       
    
    #Cada dfStocks é um dataframe com o produto específico
    dfStocks=dfDelta[dfDelta["DESC_ARTIGO"]==coluna][["id", "Semana", "STORE", "SOH", "PRES_STOCK", "INTRANSIT", "EXPECTED", "Dia_Actual"]]   #Dados Ninjas, seleccionar colunas do café específico
    
    #Juntar dados de acordo com o dia e a loja
    dfMeio=pd.merge(dfMeio, dfStocks, how="left", on = ["id","STORE", "Semana"])  
    
    #Mudar nomes para ser adaptado a cada produto
    dfMeio=dfMeio.rename(columns={"SOH": "STOCK %s" % coluna, "PRES_STOCK":"PRESLINEAR %s" % coluna, "INTRANSIT":"INTRANSIT %s" % coluna, "EXPECTED":"EXPECTED %s" % coluna, "Dia_Actual":"SELLOUT %s" % coluna})                       #Nomear coluna nova
    
    

- Quantos produtos estão presentes? <span style="color:red">Vale a pena? </span>

In [33]:
# Criar coluna com o número dos produtos em causa presentes no instante

dfMeio["Num_Produtos"] = dfMeio[produtos].sum(axis=1)

dfMeio.loc[(dfMeio["Num_Produtos"] == 0) & (dfMeio[produtos[1]].isna()), "Num_Produtos"] = np.nan

In [34]:
dfFinal=dfMeio.copy()

# Já temos DataFrame com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição

## Fazer:

- Rotura
- Sinal -> Avaliação da informação ninja vs stock
- Ciclos
- Adequação de Stock

> Rotura

In [165]:
for i in produtos:
    rotura  = "ROTURA %s" % i
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    
    dfFinal[rotura] = np.where((dfFinal[stock] > 0) & ~(dfFinal[i].isna()), 0, 
                      np.where((dfFinal[stock] <= 0) & (dfFinal[preslinear] > 0) & ~(dfFinal[i].isna()), 1, 
                      np.nan))


> Pré-Rotura

In [166]:
for i in produtos:
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    pre_rotura= "PRÉ-ROTURA %s" % i
    
    dfFinal[pre_rotura] = (dfFinal[stock] < dfFinal[preslinear]).astype(int)

>Sinal

In [167]:
#dfFinal["Sinal"]= pd.Series(dtype='int')

# Define the mappings of numbers to strings
mapping = {1: "Verdadeiramente Correcto",
           2: "Falso Positivo",
           3: "Falso Negativo",
           4: "Rejeição Correcta"}

for i in produtos:
    sinal = "SINAL %s" % i
    stock  = "STOCK %s" % i
    
    dfFinal[sinal] = np.where((dfFinal[stock] > 0) & (dfFinal[i] == 1), "Verdadeiramente Correcto",
                     np.where((dfFinal[stock] > 0) & (dfFinal[i] == 0), "Falso Positivo",
                     np.where((dfFinal[stock] <= 0) & (dfFinal[i] == 1), "Falso Negativo",
                     np.where((dfFinal[stock] <= 0) & (dfFinal[i] == 0), "Rejeição Correcta", np.nan))))

    # Map the numbers to the corresponding strings
    dfFinal[sinal] = dfFinal[sinal].map(mapping)
    
    

# 1: Há Stock e Sinal Positivo Ninja
# 2: Há Stock e Sinal Negativo Ninja
# 3: Não há Stock e Sinal Positivo Ninja
# 4: Não há Stock e Sinal Negativo Ninja

> Ciclos

In [168]:
for i in produtos:
    ciclos = "CICLO %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    dfFinal[ciclos]=dfFinal[stock]/dfFinal[linear]

> Adequação

In [169]:
# Define the mappings of numbers to strings
mapping = {1: "Stock Suficiente",
           2: "Stock Insuf c Forn Adequado",
           3: "Stock Insuf c Forn Desadequado"}

for i in produtos:
    adequa = "ADEQUAÇÃO %s" % i
    ciclos = "CICLO %s" % i
    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    
    dfFinal[adequa] = np.where(dfFinal[ciclos] > 1.1, "Stock Suficiente",
                      np.where((dfFinal[ciclos] <= 1.1) & (dfFinal[transito] + dfFinal[esperado] + dfFinal[stock] >= dfFinal[linear]), "Stock Insuf c Forn Adequado",
                      np.where((dfFinal[ciclos] <= 1.1) & (dfFinal[transito] + dfFinal[esperado] + dfFinal[stock] < dfFinal[linear]), "Stock Insuf c Forn Desadequado", 
                      np.nan)))
    
    # Map the numbers to the corresponding strings
    dfFinal[adequa] = dfFinal[adequa].map(mapping)

- Dias para a rotura

In [170]:
for i in produtos:
    stock  = "STOCK %s" % i
    sellout = "SELLOUT %s" % i
    dias_rotura = "Dias_para_Rotura %s" % i
    
    dfFinal[dias_rotura]=dfFinal[stock]/dfFinal[sellout]

- Dias para a rotura de prateleira

In [171]:
for i in produtos:
    linear = "PRESLINEAR %s" % i
    sellout = "SELLOUT %s" % i
    dias_rotura_prat = "Dias_para_Rotura_Linear %s" % i
    
    dfFinal[dias_rotura] = dfFinal[linear] / dfFinal[sellout]

- Balanço

In [172]:
for i in produtos:
    balanço = "BALANCE %s" % i
    stock  = "STOCK %s" % i
    sellout = "SELLOUT %s" % i
    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    
    dfFinal[balanço] = dfFinal[sellout] / (dfFinal[stock] + dfFinal[transito] + dfFinal[esperado])

# Acabamos com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição
- Rotura
- Pré-Rotura
- Sinal 
- Ciclos
- Adequação de Stock
- Dias para a rotura normal
- Dias para a rotura móvel XXXXXXXXXXX
- Dias para a rotura de prateleira
- Balanço

# Escrever

In [37]:
escrever_excel(dfFinal, "OUT2DataFusion2022")
#dfFinalCorr.to_csv('ficheiro_NSS.csv', index=False)